In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras as tf_keras

In [2]:
with open("data_files/nietzsche.txt", "rt") as f: # rt: read text
    nietzche_text = f.read()

In [3]:
print( len(nietzche_text) ) # 600893자
nietzche_text[:30]  # 대소문자 모두 포함

600893


'PREFACE\n\n\nSUPPOSING that Truth'

In [4]:
# 대문자 -> 소문자
nietzche_lower_text = nietzche_text.lower()
nietzche_lower_text[:30]

'preface\n\n\nsupposing that truth'

In [5]:
# 전체 텍스트에 포함된 문자 확인
print( np.unique(list(nietzche_lower_text)) )
print( np.unique(list(nietzche_lower_text)).shape )

['\n' ' ' '!' '"' "'" '(' ')' ',' '-' '.' '0' '1' '2' '3' '4' '5' '6' '7'
 '8' '9' ':' ';' '=' '?' '[' ']' '_' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i'
 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' 'ä'
 'æ' 'é' 'ë']
(57,)


In [6]:
# 문자 사전 만들기

set(nietzche_lower_text)
# set : 중복되지 않는 리스트
sorted_chars = sorted(set(nietzche_lower_text))
print( "단어 집합 >", sorted_chars )

# 정렬된 문자 앞부터 0으로 메기기 시작
char_to_idx = { ch:i for i, ch in enumerate(sorted_chars) } # 문자 : 숫자
print( "문자:숫자 >", char_to_idx )

idx_to_char = { i:ch for ch, i in char_to_idx.items() }     # 숫자 : 문자 (char_to_idx를 반전)
print( "숫자:문자 >", idx_to_char )

단어 집합 > ['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ä', 'æ', 'é', 'ë']
문자:숫자 > {'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '=': 22, '?': 23, '[': 24, ']': 25, '_': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52, 'ä': 53, 'æ': 54, 'é': 55, 'ë': 56}
숫자:문자 > {0: '\n', 1: ' ', 2: '!', 3: '"', 4: "'", 5: '(', 6: ')', 7: ',', 8: '-', 9: '.', 10: '0', 11: '1', 12: '2', 13: '3', 14: '4', 15: '5', 16: '6', 17: '7', 18: '8', 19: '9', 20: ':', 21:

In [7]:
# 학습 내용

# n개의 연속된 문자 -> n+1번째 문자 예측
# (입력데이터)         (출력데이터, target)
# n개의 연속된 문자를 이용해서, n+1번째 1개의 문자 예측하기
# 단어라면 함축적 의미를 가지니 여려군데서 의미 달라질 수 있어서 임베딩처리가 마땅하지만,
# 문자는 임베딩처리하지 않고 단어 사전 수준에서 작업
# 문자가 단어사전 수준으로 하더라도 속성이 57개 뿐이라 가능하겠군.
# 원핫인코딩 단어사전을 가지고 작업할 예정이다.

In [8]:
# 학습 데이터 준비

sequence_length = 50    # 연속된 문자의 개수
step = 3                # stride (3문자씩 이동하면서 데이터 추출) : 다음 입력 데이터를 만들 때 3개 띈다

sequences = []        # (batch_size, 입력문자개수, 단어사전크기)
next_chars = []         # (batch_size, 단어사전크기) 한 개 예측 되는 문자

# 50개 + 1개 > 51개
for idx in range(0, len(nietzche_lower_text) - sequence_length, step):
    sequences.append(nietzche_lower_text[idx:idx+sequence_length])
    next_chars.append(nietzche_lower_text[idx+sequence_length]) # 다음 예측할 문자

# print( len(sequences), len(next_chars) )
# print( sequences[0], next_chars[0] )
# 입력 : supposing that truth is a woman--what th, 출력 : e > 숫자로 바꿔야한다.

X = np.zeros(shape=(len(sequences), sequence_length, len(sorted_chars)))
# 데이터를 다 0으로 채워넣고 문자있으면 반복문 돌면서 1로 체크
y = np.zeros(shape=(len(sequences), len(sorted_chars)))

for si, sequence in enumerate(sequences):   # si : 입력 문장 순서 번호
    # print(si, sequence)
    for ci, ch in enumerate(sequence):      # ci : 한 개의 입력 문장 안의 문자 순서 번호
        X[si, ci, char_to_idx[ch]] = 1      # 문장 번호 안에서 각 문자를 돌면서, 있는 문자에 1을 넣는다
        y[si, char_to_idx[next_chars[si]]] = 1
        # print(ci, ch, end=", ")
    # if si == 2:
    #     break

In [ ]:
# 모델 구조 설계 (순환 신경망)

input = tf_keras.layers.Input(shape=(sequence_length, len(sorted_chars)))
x = tf_keras.layers.LSTM(units=128)(input)
output = tf_keras.layers.Dense(units=len(sorted_chars), activation='softmax')(x)
# 회귀 아닌 분류. 그래서 마지막 유닛수는 분류하는 카테고리 수 만큼 (단어사전 크기)
model = tf_keras.models.Model(input, output)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50, 57)]          0         
                                                                 
 lstm (LSTM)                 (None, 128)               95232     
                                                                 
 dense (Dense)               (None, 57)                7353      
                                                                 
Total params: 102585 (400.72 KB)
Trainable params: 102585 (400.72 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
# 모델 학습 설계
model.compile(loss='categorical_crossentropy', 
              optimizer=tf_keras.optimizers.Adam(learning_rate=0.01),   # 학습률 좀 더 높여도 괜찮 0.01>0.1
              metrics=['accuracy'])

In [52]:
history = model.fit(X, y, batch_size=128, epochs=50)    # 에폭 더 늘려도 괜찮

Epoch 1/50
1565/1565 [==============================] - 79s 49ms/step - loss: 1.6738 - accuracy: 0.4954
Epoch 2/50
1565/1565 [==============================] - 71s 45ms/step - loss: 1.5850 - accuracy: 0.5157
Epoch 3/50
1565/1565 [==============================] - 75s 48ms/step - loss: 1.5452 - accuracy: 0.5253
Epoch 4/50
1565/1565 [==============================] - 78s 50ms/step - loss: 1.5157 - accuracy: 0.5341
Epoch 5/50
1565/1565 [==============================] - 75s 48ms/step - loss: 1.4953 - accuracy: 0.5392
Epoch 6/50
1565/1565 [==============================] - 74s 47ms/step - loss: 1.4761 - accuracy: 0.5449
Epoch 7/50
1565/1565 [==============================] - 72s 46ms/step - loss: 1.4616 - accuracy: 0.5492
Epoch 8/50
1565/1565 [==============================] - 72s 46ms/step - loss: 1.4500 - accuracy: 0.5521
Epoch 9/50
1565/1565 [==============================] - 71s 45ms/step - loss: 1.4382 - accuracy: 0.5558
Epoch 10/50
1565/1565 [==============================] - 75s 48m

In [ ]:
# 모델 저장
model.save('models/generation_model.keras')

In [24]:
# 모델 불러오기
loaded_model = tf_keras.models.load_model('models/generation-model.keras')

In [25]:
# D:\Work\workspace\dl-basic\research.ipynb 참고
def select_character(preds, temperature=1.0): 
    # temperature 값이 작을 수록 낮은 확률의 값이 선택 가능성이 낮짐 preds: softmax결과값 사용
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature     # log를 활용해서 차이의 순서는 유지하되, 폭을 줄임
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1) # 주어진 확률에 따라 다음 값 랜덤 선택 (랜덤 샘플링)
    return np.argmax(probas)

In [30]:
# argmax()
start_idx = np.random.randint(0, len(nietzche_lower_text) - sequence_length)
seed_text = nietzche_lower_text[start_idx:start_idx + sequence_length]
full_text = seed_text

print(seed_text)
print('=' * 50)
for idx in range(100):
    # 입력 데이터 만들기
    sample = np.zeros(shape=(1, sequence_length, len(sorted_chars)))    # 1, 50, 57
    for ci, c in enumerate(seed_text):
        sample[0, ci, char_to_idx[c]] = 1
    
    predicted_values = model.predict(sample, verbose=0)
    selected_char_idx = predicted_values[0].argmax()
    full_text += sorted_chars[selected_char_idx]
    seed_text = full_text[idx+1:]

    print(sorted_chars[selected_char_idx], end="")

ful in dealing with those who attach great importa
 l) doo000((,,;aaaauaauajauajf"kf"kkf;66t4v]u=éd'ggg:8g:8g:8g::8dggg:g:8g::8m dggggg::8me  dggggg:8g

In [31]:
# 온도 직접 지정
start_idx = np.random.randint(0, len(nietzche_lower_text) - sequence_length)
seed_text = nietzche_lower_text[start_idx:start_idx + sequence_length]
full_text = seed_text

print(seed_text)
print('=' * 50)
for idx in range(100):
    # 입력 데이터 만들기
    sample = np.zeros(shape=(1, sequence_length, len(sorted_chars)))    # 1, 50, 57
    for ci, c in enumerate(seed_text):
        sample[0, ci, char_to_idx[c]] = 1
    
    predicted_values = model.predict(sample, verbose=0)
    selected_char_idx = select_character(predicted_values[0], 1)
    full_text += sorted_chars[selected_char_idx]
    seed_text = full_text[idx+1:]

    print(sorted_chars[selected_char_idx], end="")

tify, the less likely he is to look directly at th
tgh x.-! b'!cd_uë"!--e0c.hp]'t f-abj,9 u.!04jæxd7ë9=vskä8,:nva(u:qb=.fmml
1br,ogwh4hftc ]i-;iæ'_épv3

In [33]:
# temperature로 온도 범위 설정 + 저장된 모델(선생님 모델) 불러오기
start_idx = np.random.randint(0, len(nietzche_lower_text) - sequence_length)
seed_text = nietzche_lower_text[start_idx:start_idx + sequence_length]

for temperature in [0.5, 1.0, 1.5]:

    full_text = seed_text   # full_text 초기화 하기

    print("temperature : {0}".format(temperature))
    print(seed_text, end="")
    for idx in range(100):
        # 입력 데이터 만들기
        sample = np.zeros(shape=(1, sequence_length, len(sorted_chars)))    # 1, 50, 57
        for ci, c in enumerate(seed_text):
            sample[0, ci, char_to_idx[c]] = 1
        
        predicted_values = loaded_model.predict(sample, verbose=0)

        selected_char_idx = select_character(predicted_values[0], temperature)

        full_text += sorted_chars[selected_char_idx]
        seed_text = full_text[idx+1:]

        print(sorted_chars[selected_char_idx], end="")

    print('\n\n')
    print("=" * 50)

temperature : 0.5
 englishman
have done by means of a stronger digestiv

C:\Users\human\AppData\Local\Temp\ipykernel_7828\1031459413.py:5: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature     # log를 활용해서 차이의 순서는 유지하되, 폭을 줄임


e process them in the promise, and the consequently the original observation of former such meter


temperature : 1.0
ntly the original observation of former such meter alone or just
(deckerate
shallowest individuals of show the escry, when facrively the domien to the


temperature : 1.5
f show the escry, when facrively the domien to the in
so feetiracal mnegict, sot?
spire of respectarily step men. unnou mustines--is
over furbst own p


